In [ ]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate

In [1]:
import warnings
warnings.filterwarnings('ignore')

# 設定 APIKEY
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# **Intorduction to 🤗 Hugging Face and Transformers Library**

[🤗 hugging face NLP course](https://huggingface.co/learn/nlp-course/chapter1/1)


NLP 是語言學和機器學習領域，專注於理解與人類語言相關的一切。 NLP 任務的目標不僅是單獨理解單字，而且能夠理解這些單字的上下文。</p>
NLP 常見的任務有：
   1. **Classifying whole sentences 文本分類（分類整句）**：將整個句子進行分類
      - 情感分析：「這部電影很棒！」 -> positive
      - 垃圾郵件檢測：「你的 iphone 已被嚴重損壞」-> spam 
   2. **Classifying each word in a sentence 單詞分類**：對一句話中的所有字進行分類
      - 語法分析：「他跑得快」-> 他（代詞）跑（動詞）得（副詞）快（形容詞）
      - 命名實體 NER
   3. **Sentence Generation**
      1. **填充遮蔽詞**：「像這種要求，我這輩子[mask]！」-> [mask] 預測為 沒聽過
      2. **自動生成**：「今天天氣如何」-> 「今天天氣非常晴朗適合外出」
      3. **翻譯**：「你好」-> 「Hello」
      4. **摘要（問答）**
         1. 從文本中提取答案 Extractive QA：「法國的首都是巴黎。   首都在哪」-> 巴黎（藉由巴黎在原文的 index 抓出來）
         2. 以生成模型進行摘要 Generative QA：「法國的首都是巴黎。   首都在哪」-> 首都在巴黎（使用生成模型，例如 chatgpt）

## **`pipeline` in Hugging Face transformers**
``transformers`` 為 🤗Hugging face 提供的套件，讓開發者可以創建、使用 Hugging face hub 上 NLP、LLM 的模型</p>
> Hugging face hub 上的模型不只有 transformer，任何人都可以上傳任何類型的模型或資料集

在 `transformers` 中最高階的函數是 `pipeline`，
該函數將使用模型需要的預處理、推理與後處理串連起來，</p>
傳入指定的 task，`pipeline` 會自動以適合的模型進行推理（預測）。

可以從 [hub](https://huggingface.co/models) 透過 Tasks、Languages 篩選找到自己想要應用的模型</p>
從 1. [task summary](https://huggingface.co/docs/transformers/task_summary) 2. [Tasks](https://huggingface.co/tasks) 找到支援的 NLP 相關任務

### **使用 pipeline 完成常見 NLP 任務**

1. 情感分析 aka 分類問題

In [3]:
from pprint import pprint
from transformers import pipeline
pipe = pipeline(task="sentiment-analysis")
pipe("this is awesome!!!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998723268508911}]

2. 命名實體</p>



在 ner 任務中，模型會對所有字詞（token） 進行分類，得到：</p>
1. 該字詞（token）對應的 entity
2. score 機率值
3. 以及對應到文本的起始結束位置

In [ ]:
ner_pipe = pipeline(task="ner", 
                # model='dslim/bert-base-NER'
                )
ner_pipe("Hugging Face is a French company based in New York City.")

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [2]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad, 
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

### **利用 Conversation class 與 text-generation model 實作 chatbot**

In [3]:
from torch import cuda, bfloat16
from transformers import pipeline
from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cpu


因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的同仁可以參考 Huggingface 官方文件。</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`

In [ ]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

In [14]:
chatbot = pipeline(
    "text-generation", 
    model=hf_model, 
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

聊天式模型，例如 ChatGPT 其實基本上是透過 text-generation 作為基礎模型，進一步訓練模型能過聊天。
所以模型的選擇，我們可以在 Huggingface 上找到 text-generation 任務的模型，應該都可以支援。</p>

比較特別的是，要做聊天任務時，模型需要一些特殊符號來區別每一段訊息是來自於 User 或是 AI 還是 System Prompt</p>
而各個模型的特殊符號不盡相同，需要去查閱官方文件。例如 Demo 使用的聯發科 Breeze 模型是透過 `[INST]` 、 `[/INST]` 以及 `<s>` 作為區隔。
所以我們在使用模型時，就會需要將文字加上這些特殊符號才能夠發揮模型聊天的能力。</p>

通常我們會使用 list of dict 的方式處存聊天的記錄，使用 role 區別 user 與 ai，content 代表內容，而 Hugging face 的模型也支援這樣的格式，例如：


```
[
    {"role": "user", "content": "嗨你好嗎"},
    {"role": "assistant", "content": "嗨您好，我是您的 AI 助理，很高興為您服務。"},
    {"role": "user", "content": "掰掰"},
    {"role": "assistant", "content": "掰掰，期待再相見"},

]

```

我們可以透過實例化 Conversation 這個 class，透過 `add_user_input` 與 `append_response` 新增歷史使用者輸入與模型回覆，將資料變為上述的資料格式再送給模型進行推理。


In [1]:
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

In [16]:
conversation.add_user_input("provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?")

In [17]:
chatbot_result = chatbot(conversation.messages)[-1]['generated_text']
conversation.append_response(chatbot_result)

In [18]:
conversation.add_user_input("那什麼是 bert?")
chatbot_result = chatbot(conversation.messages)[-1]['generated_text']

In [19]:
chatbot_result

' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google的一个自然语言处理模型。它基于Transformers架构，可以对文本数据学习语义表达，用于各种自然语言处理任务，如情感分析、命名实体识别、情况语言模型等。Bert模型通过预训练在大量的非任务专用数据上获得知识，然后在特定的任务上进行微调。'

### **Practi**

### **embedding model (feature extraction)**
[參考](https://huggingface.co/tasks/feature-extraction)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model="facebook/bart-base")

In [26]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

/opt/conda/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
query = "為什麼 ML 需要做正規化"

source_sentence = [
    'Regularization is important!',
    'Dropout is important!',
    'Missing Data Handling is important!'
]

In [30]:
import numpy as np
def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [36]:
most_related_sentence = None
max_similarity = 0

for sentence in source_sentence:
    sim = calculate_cosine_similarity(
    embedding_model.encode(query),
    embedding_model.encode(sentence)
    )
    
    if sim > max_similarity:
        most_related_sentence = sentence
        max_similarity = sim
        
    print(f"{query} vs {sentence} similarity: {sim}")
    
print(f"與 Query 最相關文本：{most_related_sentence}")

為什麼 ML 需要做正規化 vs Regularization is important! similarity: 0.8665976524353027
為什麼 ML 需要做正規化 vs Dropout is important! similarity: 0.8055095672607422
為什麼 ML 需要做正規化 vs Missing Data Handling is important! similarity: 0.8144524693489075
與 Query 最相關文本：Regularization is important!
